### Create training dataset
In this notebook We are going to create training datasets and register to Hopsworks Feature Store. This training dataset will be later used to train for graph emmbedings model.
![Training Dataset](./images/create_training_dataset.png)

In [1]:
spark

Starting Spark application


ID,Application ID,Kind,State,Spark UI,Driver log
3,application_1651179967574_0004,pyspark,idle,Link,Link


SparkSession available as 'spark'.

In [2]:
import hashlib
from datetime import datetime
from pyspark.sql import functions as F
from pyspark.sql.types import FloatType
import hsfs

### Instantiate a connection and get the project feature store handler 

In [3]:
# Create a connection
connection = hsfs.connection()
# Get the feature store handle for the project's feature store
fs = connection.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

### Get transactions feature group handle

In [4]:
transactions_fg = fs.get_feature_group("transactions_fg", 1)
transactions_fg.show(5)

+--------------+--------+--------+-------+-------+--------+
|tran_timestamp|  source|  target|tran_id|tx_type|base_amt|
+--------------+--------+--------+-------+-------+--------+
|        Aug-18|35d9e14d|ea20f3ea| 328889|      4|  341.23|
|        Aug-18|ecaba057|42e760b1| 328890|      4|  846.74|
|        Aug-18|43e1ce2b|d85de509| 328891|      4|  269.45|
|        Aug-18|2d76caaa|9dff007b| 328892|      4|  180.32|
|        Aug-18|b2efe3c6|92cf8d33| 328893|      4|  507.09|
+--------------+--------+--------+-------+-------+--------+
only showing top 5 rows

In [5]:
transactions_fg.read().count()

438386

### Load alert transactions feature group  handle

In [6]:
alert_transactions_fg = fs.get_feature_group("alert_transactions_fg", 1)
alert_transactions_fg.show(5)

+--------------+--------+------+-------+
|    alert_type|alert_id|is_sar|tran_id|
+--------------+--------+------+-------+
|scatter_gather|      19|     1| 180790|
|scatter_gather|      19|     1| 182036|
|scatter_gather|      28|     1| 182118|
|scatter_gather|      30|     1| 184414|
|scatter_gather|      30|     1| 191602|
+--------------+--------+------+-------+
only showing top 5 rows

In [7]:
alert_transactions_fg.read().count()

915

### Load party feature group  handle

In [8]:
party_fg = fs.get_feature_group("party_fg", 1)
party_fg.read().show()

+--------+----+
|      id|type|
+--------+----+
|9ad5bb7e|   0|
|9ad7aece|   1|
|9adc0252|   1|
|9adc6316|   1|
|9aef2e25|   0|
|9af3d1e4|   0|
|9b0d0ebc|   1|
|9b14d03a|   1|
|9b304b65|   1|
|9b31dd13|   0|
|9b34d454|   1|
|9b3e392a|   0|
|9b42b3ed|   1|
|9b48182e|   1|
|9b4f5d8f|   0|
|9b52f33e|   1|
|9b5f2caa|   0|
|9b6239bf|   0|
|9b66e49c|   1|
|9b6985f6|   1|
+--------+----+
only showing top 20 rows

## Create training datasets
To create training datasest we will use hsfs `Query` object. Training dataset's metadata, created from hsfs `Query` object, contains information such as: 
* which feature groups it was created from; 
* commit id of these feature froups;
* the order of features. 

This will give us possibility to 
* track back and see which feature were used to create this training; 
* perform time-travel and see how features looked like when this training dataset was created ;
* reconstruct feature order during model inferencing.     

### Create graph edge training datasets 

In [9]:
edges = transactions_fg.select(["source","target","tran_id","tx_type","base_amt"])

In [10]:
edges.read().show()

+--------+--------+-------+-------+--------+
|  source|  target|tran_id|tx_type|base_amt|
+--------+--------+-------+-------+--------+
|35d9e14d|ea20f3ea| 328889|      4|  341.23|
|ecaba057|42e760b1| 328890|      4|  846.74|
|43e1ce2b|d85de509| 328891|      4|  269.45|
|2d76caaa|9dff007b| 328892|      4|  180.32|
|b2efe3c6|92cf8d33| 328893|      4|  507.09|
|94405fbd|5033b863| 328899|      4|   404.6|
|ecc461e1|7b0a5b49| 328900|      4|  903.96|
|0c535264|c316b5f7| 328902|      4|  805.33|
|c32b2ef9|0ba25b05| 328903|      4|  263.29|
|896eaf11|cd7dbc4f| 328905|      4|  220.48|
|35ca5309|fafb6f18| 328907|      4|  981.35|
|c6d05e8e|8dd827b1| 328909|      4|  141.77|
|40c671aa|faa1e4dc| 328910|      4|  674.86|
|d3adb450|fbc5ada0| 328919|      4|  351.92|
|65ec7365|7b870ef2| 328925|      4|  463.31|
|9405082c|d9768d01| 328927|      4|  647.38|
|5a99160f|a87a051f| 328929|      4|  320.18|
|ff26d5c9|3858d176| 328933|      4|  351.77|
|7e90e454|29342e2b| 328935|      4|  830.61|
|397d6734|

In [11]:
edges.read().count()

438386

In [12]:
edges_td_meta = fs.create_training_dataset(name="edges_td",
                                           version=1,
                                           data_format="csv",
                                           description="edges training dataset",
                                           coalesce=True,
                                           statistics_config={"enabled": True, "histograms": True, "correlations": False}
                                          )
edges_td_meta.save(edges)

### Create graph node training dataset

In [13]:
nodes = party_fg.select_all()
nodes.read().show()

node_td_meta = fs.create_training_dataset(name="node_td",
                                          version=1,
                                          data_format="csv",   
                                          description="node training dataset",
                                          statistics_config={"enabled": True, "histograms": True, "correlations": False},
                                          coalesce=True)
node_td_meta.save(nodes)

+--------+----+
|      id|type|
+--------+----+
|9ad5bb7e|   0|
|9ad7aece|   1|
|9adc0252|   1|
|9adc6316|   1|
|9aef2e25|   0|
|9af3d1e4|   0|
|9b0d0ebc|   1|
|9b14d03a|   1|
|9b304b65|   1|
|9b31dd13|   0|
|9b34d454|   1|
|9b3e392a|   0|
|9b42b3ed|   1|
|9b48182e|   1|
|9b4f5d8f|   0|
|9b52f33e|   1|
|9b5f2caa|   0|
|9b6239bf|   0|
|9b66e49c|   1|
|9b6985f6|   1|
+--------+----+
only showing top 20 rows

## create derived feature group `alert_nodes_fg`, whether nodes were part of previously known money laundering scheme or not

In [14]:
alert_edges = transactions_fg.select(["source","target","tran_id","tx_type","base_amt"]).join(alert_transactions_fg.select(["is_sar"]),["tran_id"],"left")
alert_edges = alert_edges.read().where(F.col("is_sar")==1)
alert_sources = alert_edges.select(["source"]).toDF("id")
alert_targets = alert_edges.select(["target"]).toDF("id")
alert_nodes = alert_sources.union(alert_targets).dropDuplicates(subset=["id"])
alert_nodes = alert_nodes.withColumn("is_sar",F.lit(1))
alert_nodes.cache()
alert_nodes.show()

+--------+------+
|      id|is_sar|
+--------+------+
|43e028ef|     1|
|33a8ff5b|     1|
|fcf3bbf3|     1|
|8b9017b8|     1|
|65636b63|     1|
|d73e5230|     1|
|9c187eed|     1|
|68c0230d|     1|
|550a25ff|     1|
|c0be245b|     1|
|963b978f|     1|
|cdbd2ed5|     1|
|84563a83|     1|
|da77c74b|     1|
|840701de|     1|
|dc37f73b|     1|
|b0f4351c|     1|
|c29d75dc|     1|
|dd2ebcf1|     1|
|d7c99aa5|     1|
+--------+------+
only showing top 20 rows

In [15]:
alert_sources = alert_edges.select(["source"]).toDF("id")
alert_targets = alert_edges.select(["target"]).toDF("id")
alert_nodes = alert_sources.union(alert_targets).dropDuplicates(subset=["id"])
alert_nodes = alert_nodes.withColumn("is_sar",F.lit(1))
alert_nodes.cache()
alert_nodes.show()

+--------+------+
|      id|is_sar|
+--------+------+
|43e028ef|     1|
|33a8ff5b|     1|
|fcf3bbf3|     1|
|8b9017b8|     1|
|65636b63|     1|
|d73e5230|     1|
|9c187eed|     1|
|68c0230d|     1|
|550a25ff|     1|
|c0be245b|     1|
|963b978f|     1|
|cdbd2ed5|     1|
|84563a83|     1|
|da77c74b|     1|
|840701de|     1|
|dc37f73b|     1|
|b0f4351c|     1|
|c29d75dc|     1|
|dd2ebcf1|     1|
|d7c99aa5|     1|
+--------+------+
only showing top 20 rows

In [16]:
alert_nodes_df = nodes.read().join(alert_nodes,["id"], "left").withColumn("is_sar",F.when(F.col("is_sar") == 1, F.col("is_sar")).otherwise(0))
alert_nodes_df.cache()
alert_nodes_df.show()

+--------+----+------+
|      id|type|is_sar|
+--------+----+------+
|9ad5bb7e|   0|     0|
|9ad7aece|   1|     0|
|9adc0252|   1|     0|
|9adc6316|   1|     0|
|9aef2e25|   0|     0|
|9af3d1e4|   0|     0|
|9b0d0ebc|   1|     0|
|9b14d03a|   1|     0|
|9b304b65|   1|     0|
|9b31dd13|   0|     0|
|9b34d454|   1|     0|
|9b3e392a|   0|     1|
|9b42b3ed|   1|     0|
|9b48182e|   1|     0|
|9b4f5d8f|   0|     0|
|9b52f33e|   1|     0|
|9b5f2caa|   0|     0|
|9b6239bf|   0|     0|
|9b66e49c|   1|     0|
|9b6985f6|   1|     0|
+--------+----+------+
only showing top 20 rows

In [17]:
alert_nodes_df.where(F.col("is_sar") == 1).count()

816

In [18]:
alert_nodes_df.where(F.col("is_sar") == 0).count()

6531

In [19]:
alert_nodes_fg = fs.create_feature_group(name="alert_nodes_fg",
                                       version=1,
                                       primary_key=["id"],
                                       description="node embeddings from transactions, derived fg",
                                       time_travel_format="HUDI",     
                                       online_enabled=True,                                                
                                       statistics_config=False)

alert_nodes_fg.save(alert_nodes_df)

### Training datasets exploration from the user interface

##### Hopsworks provides user interface that enables to discover and explore avaibale Training datasets and related features. Bellow screen shot demonstrates how one can preview list of available features in `edges_td` and get basic information such us identify feature types and which one is as a label.   

![Incremental Feature Engineering](./images/td_features.png)

##### One of the important steps of training dataset exploration is discover distribution of ist features. Since we enabled statistics to be computed durring training dataset creation we can easily preview descriptive statitsics. If training dataset has splits we can also preview statitics for each split separately and campare distributions to make sure that train and test splits have similar distibutions.   


![Incremental Feature Engineering](./images/td_stats.png)

##### Hopsworks UI also give access to training dataset activity timeline metadata.  
![Incremental Feature Engineering](./images/td_activity.png)